# SF Crime Analysis: Ensemble Model

### Data-x (IEOR 135)


In [1]:
# Load required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Read in data
df1 = pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
#df2 = pd.read_csv('311_Cases.csv')

In [3]:
# Format date columns
df1['Date'] = pd.to_datetime(df1['Date'])
df1['Year-Month'] = df1['Date'].map(lambda x: '{year}-{month}'.format(year=x.year,
                                                              month=x.month,
                                                            day=x.day))
df1['Year'] = df1['Date'].dt.year.astype(int)
df1['Week_Number'] = df1['Date'].dt.week
df1['Month_Number'] = df1['Date'].dt.month

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose

### By Month

In [5]:
# Splice relevant columns
dfDecompose = df1[['IncidntNum','Category','Year','Week_Number', "Month_Number", 'Date']]

# Select ASSAULT crimes only
dfDecompose = dfDecompose[dfDecompose['Category'] == 'ASSAULT']

# Drop records from 2018 since data from only part of the year is available
dfDecompose = dfDecompose[(dfDecompose['Year'] < 2018)]

# Group by month
monthGrouped = dfDecompose.resample('M', on='Date')[['IncidntNum']].count() #.reset_index().sort_values(by='Date')
monthGrouped.rename(columns = {'IncidntNum': "Incidents/Month"}, inplace = True)

# Forecasting Models

## Error Metrics to Compare Models

The models will be compared with the following metrics. Models are tested on data from 2003  - 2016 and then tested on 2017 data.

**Mean Absolute Error (MAE)** <br>
$Error =  mean( |y_{actual} - y_{predict}|)$

**Root Mean Squared Error** <br>
$Error =  \sqrt{mean((y_{actual} - y_{predict})^{2})}$

**Mean Abosolute Percentage Error (MAPE)** <br>
$Error =  mean(|\frac{y_{actual} - y_{predict}}{y_{actual}}|)$

In [6]:
#Functions to quantify forecast error
# Referenced from https://otexts.com/fpp2/accuracy.html

# Mean Absolute Error
def mae(y_predict, y_actual):
    return round(np.mean(abs(y_predict - y_actual)),2)

# Mean Root Squared Error
def rmse(y_predict, y_actual):
    return round(np.sqrt(np.mean((y_predict - y_actual)**2)),2)

# Mean Absolute Percentage Error
def MAPE(y_predict, y_actual):
    return round(np.mean(abs((y_predict - y_actual)/y_actual)) * 100,2)

def print_error(predictedValues, actualValues):
    print("OUT OF SAMPLE ERROR:") 
    print("Mean Absolute Error (MSE): ", mae(predictedValues, actualValues))
    print("Root Mean Squared Error (RSME): ", rmse(predictedValues, actualValues))
    print("Mean Absolute Percentage Error (MAPE): ", MAPE(predictedValues, actualValues), '%')

## Build Training Dataset

In [7]:
# Data we are using to forecast
monthGrouped

,Incidents/Month
Date,
2003-01-31,1174
2003-02-28,1050
2003-03-31,1231
2003-04-30,1016
2003-05-31,1117
...,...
2017-08-31,1293
2017-09-30,1188
2017-10-31,1183


In [8]:
train = monthGrouped[monthGrouped.index.year < 2017]
test = monthGrouped[monthGrouped.index.year == 2017]['Incidents/Month']

## SARIMA Model

Analysis steps and code from https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

Seasonal Auto Regressive Integrated Moving Average (ARIMA) models are linear regression models built upon lags (different between current and previous obsevation) and forecast errors.

ARIMA model parameters:
- p is the order of the AR term (the number of lags to be used as predictors)
- q is the order of the MA term (then number of lagged forecast errors to be used as predictors)
- d is the number of differencing required to make the time series stationary


Code adapted from https://github.com/liyenhsu/SF-Crime-Analysis/blob/master/sf_crime_analysis.ipynb

In [9]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import itertools
warnings.filterwarnings("ignore")

In [10]:
def sarima_model(train, steps):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]
    aic_min = float("inf")
    param = (0,0,0,0,0,0)
    best_model = None

    for x1 in pdq:
        for x2 in seasonal_pdq:
            try:
                mod = SARIMAX(train,
                              order = x1,
                              seasonal_order = x2,
                              enforce_stationarity = False,
                              enforce_invertibility = False)
                results = mod.fit()
                if results.aic < aic_min:
                    aic_min = results.aic
                    param = x1 + x2
                    best_model = mod
            except:
                continue
            
    results = best_model.fit()
    pred_test = results.get_forecast(steps=steps).predicted_mean
    return pred_test

In [11]:
sarima_model(train, 12)

2017-01-31    1169.037887
2017-02-28    1100.222483
2017-03-31    1216.335264
2017-04-30    1168.424798
2017-05-31    1206.697756
2017-06-30    1155.422937
2017-07-31    1162.419823
2017-08-31    1184.491681
2017-09-30    1230.416168
2017-10-31    1248.051253
2017-11-30    1129.638917
2017-12-31    1090.291356
Freq: M, dtype: float64

## Triple Exponential Smoothing Model

In [12]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing


def triple_exp_smoothing_model(train, steps):
    # create class
    model = ExponentialSmoothing(train, 
                                 trend = 'add', 
                                 seasonal = 'add',
                                 seasonal_periods = 12)

    # fit model
    model_fit = model.fit()

    # make prediction
    pred_test = model_fit.forecast(steps)
    
    return pred_test

In [13]:
triple_exp_smoothing_model(train, 12)

2017-01-31    1180.389778
2017-02-28    1107.398498
2017-03-31    1223.549672
2017-04-30    1175.853951
2017-05-31    1213.915094
2017-06-30    1162.580077
2017-07-31    1169.480749
2017-08-31    1191.590967
2017-09-30    1237.489955
2017-10-31    1255.187728
2017-11-30    1136.700618
2017-12-31    1097.402991
Freq: M, dtype: float64

## Random Forest Model

Analysis ideas from http://mariofilho.com/how-to-predict-multiple-time-series-with-scikit-learn-with-sales-forecasting-example/

Builds Random Forest model off of lags (current month minus previous month) and number of crimes in the previous month. 

In [14]:
# sklearn.model_selection
from sklearn.ensemble import RandomForestRegressor

In [15]:
def add_lag_prevObsers(X_rf):
    """ Creates a copy of the input dataframe and adds columns for previous crime count and lags
    """ 
    X = X_rf.copy()
    
    # Add featurs which says the current month that is being predicted
    X['MonthNum'] =  X_rf.index.month
    
    # Add number of crimes observed in the previous month
    X['Last_Month_Count'] =  X['Incidents/Month'].shift(1)
    X['Last-1_Month_Count'] =  X['Incidents/Month'].shift(2)
    X['Last-2_Month_Count'] =  X['Incidents/Month'].shift(3)
    #X['Last-11_Month_Count'] =  X['Incidents/Month'].shift(12)
    
    # Add lag columns
    for i,col in enumerate(['Last-1_Month_Count','Last-2_Month_Count']):
        X['Lag_' + str(i+1) + "_Month"] = X['Last_Month_Count'] - X[col]

    return X

### Out of Sample Testing

Calculating the OOS error is a bit difficult because to forecast one month requires the number of crime in the months before it. We can forecast one month into the future, however, if we want to forecast 2 months into the future we will need to use the forecast for the first month as an input into the 2nd months forecast.

We have used this process to forecast 12 months out. Because previous forecasts are used to calculate the next moth, the error builds with each consecutive forecast.


In [16]:
# In order to forecast, a month I will need the number of crimes observed in previous months and 
# the lag between the last month and other previous months. These functions input a DataFrame called
# y_last that has the date as the index and the return then number of lags or observations i months ago

def get_Last_Minus_i_Month_Count(y_past, i):
    return int(y_past.iloc[-(i+1),:])

def get_Lag_i_Month_Count(y_past, i):
    return get_Last_Minus_i_Month_Count(y_past, 0) - get_Last_Minus_i_Month_Count(y_past, i)
    

In [17]:
def getModelInputRow(y_input, date):
    """ In order to forecast, we need the lags and number of crimes observed in previous months. This function inputs a DataFrame
        of with index as the dates and a column with the observations and returns a row vector with each entry corresponding to 
        the previous months number of observations and lags.
    """
    # Initialize row vector that will be returned
    x_test = []
    
    # Add month
    x_test.append(date.month)
    
    # Add previous month entries
    for i in [0,1,2]:
        x_test.append(get_Last_Minus_i_Month_Count(y_input, i))
    
    # Add lag entries
    for i in [1,2]:
        x_test.append(get_Lag_i_Month_Count(y_input, i))
        
    return [x_test]

In [18]:
def prepare_split_for_machine_learning(train):
    # Add lags and umber of crimes in past months to a copy of months Grouped
    X_rfFeatures = add_lag_prevObsers(train)

    # Clean data before training model
    X_rfFeatures.dropna(inplace = True)

    #Split into X and Y training data
    X_train = X_rfFeatures.drop(columns = ['Incidents/Month'])
    y_train = X_rfFeatures['Incidents/Month']
    
    return X_train, y_train

In [19]:
def predict_on_training(random_forest_model, train, y_train, steps):
    
    # Create a copy of the number of observed crimes to append to later.
    y_hist = pd.DataFrame(y_train)

    # Get month indexes in 2017 to predict on
    months_indexes_to_forecast = triple_exp_smoothing_model(train, steps).index

    # Initialize an empty DataFrame where forecasts will be placed in
    rfForecast_df = pd.DataFrame()

    # Loop through the date indexes and forecast each date. Add that date to the dataframe and use that forecasted quantity to
    # predict the demand for the next month until the whole year has forecasts
    for date in months_indexes_to_forecast:
        nextRow = getModelInputRow(y_hist, date)
        y_pred = random_forest_model.predict(nextRow)
        forecastRow = pd.DataFrame(data = [y_pred], columns = ['Incidents/Month'], index = [date])
        y_hist = y_hist.append(forecastRow)
        rfForecast_df = rfForecast_df.append(forecastRow)
        
    # Make forecasts into a series so it ca nbe graphed
    test_pred = rfForecast_df['Incidents/Month']
    
    return test_pred

In [20]:
def random_forest_model(train, steps):
    
    X_train, y_train = prepare_split_for_machine_learning(train)
    
    # Initialize and create Random Forest Model
    RF = RandomForestRegressor()
    RF.fit(X_train, y_train)
    
    # Iteratively create forecasts. Testing the forecast into the future is difficult because each forecast depends on the 
    # number of crimes in the previous month. When forecasting 2 steps into the future, the forecast for the previous month is used
    # in the forecast for the following month. 
    
    test_pred = predict_on_training(RF, train, y_train, steps)

    return test_pred

In [21]:
random_forest_model(train, 12)

2017-01-31    1095.5
2017-02-28    1105.9
2017-03-31    1148.8
2017-04-30    1127.5
2017-05-31    1122.6
2017-06-30    1065.6
2017-07-31    1063.0
2017-08-31    1065.3
2017-09-30    1103.2
2017-10-31    1138.4
2017-11-30    1012.4
2017-12-31     960.3
Name: Incidents/Month, dtype: float64

## Adaboost Regressor

In [22]:
def adaboost_model(train, steps):
    
    X_train, y_train = prepare_split_for_machine_learning(train)
    
    # Initialize and Adaboost Regressor Model
    adaboostRegr = RandomForestRegressor()

    # Train on X_train and y_train created in the Random Forest section
    adaboostRegr.fit(X_train, y_train)
    
    # Iteratively create forecasts. Testing the forecast into the future is difficult because each forecast depends on the 
    # number of crimes in the previous month. When forecasting 2 steps into the future, the forecast for the previous month is used
    # in the forecast for the following month. 
    
    test_pred = predict_on_training(adaboostRegr, train, y_train, steps)

    return test_pred

In [23]:
adaboost_model(train, 12)

2017-01-31    1084.4
2017-02-28    1102.5
2017-03-31    1093.3
2017-04-30    1143.7
2017-05-31    1126.7
2017-06-30    1082.1
2017-07-31    1107.7
2017-08-31    1078.9
2017-09-30    1130.3
2017-10-31    1143.0
2017-11-30    1055.6
2017-12-31     958.7
Name: Incidents/Month, dtype: float64

## Gradient Boosting Regressor

In [24]:
from sklearn.ensemble import GradientBoostingRegressor

def gradient_boosting_model(train, steps):
    
    X_train, y_train = prepare_split_for_machine_learning(train)
    
    # Initialize and Adaboost Regressor Model
    gradBoostRegr = GradientBoostingRegressor()

    # Train on X_train and y_train created in the Random Forest section
    gradBoostRegr.fit(X_train, y_train)
    
    # Iteratively create forecasts. Testing the forecast into the future is difficult because each forecast depends on the 
    # number of crimes in the previous month. When forecasting 2 steps into the future, the forecast for the previous month is used
    # in the forecast for the following month. 
    
    test_pred = predict_on_training(gradBoostRegr, train, y_train, steps)

    return test_pred

In [25]:
gradient_boosting_model(train, 12)

2017-01-31    1072.554512
2017-02-28    1096.297944
2017-03-31    1073.290873
2017-04-30    1139.654215
2017-05-31    1105.838000
2017-06-30    1112.457225
2017-07-31    1089.977325
2017-08-31    1098.314328
2017-09-30    1113.004223
2017-10-31    1138.229903
2017-11-30    1053.601658
2017-12-31     964.742448
Name: Incidents/Month, dtype: float64

## Ensemble Model

In [26]:
def ensemble_predict(train, steps):
    # Run models for 12 time steps into the future
    
    print("|--", end='')
    rf = random_forest_model(train, 12)
    print("-----", end='')
    
    adabst = adaboost_model(train, 12)
    print("-------", end='')
    
    gradbst = gradient_boosting_model(train, 12)
    print("---------", end='')
    
    #sarima = sarima_model(train, 12)
    #print("SARIMA model trained!")
    
    exp_sm = triple_exp_smoothing_model(train, 12)
    print("-----------", end="")
    
    # Combine models into a dataframe
    combined_fcst = pd.concat([rf, adabst, gradbst, exp_sm], axis=1)
    combined_fcst.columns = ['rn_frst', 'adabst', 'gradbst', 'exp_sm']
    
    # Take the average forecast across all models as the estimate forecast
    combined_fcst['avg_fcst'] = combined_fcst.mean(axis = 1)
    return combined_fcst.avg_fcst

In [27]:
forecast = ensemble_predict(train, 12)

|----------------------------------